In [0]:
#imports:
import pandas as pd
import numpy as np
import math
from time import time

from sklearn.utils import shuffle

import xgboost

from scipy.stats import randint as sp_randint # int distribution for random search
from scipy.stats import uniform # float distribution for random search
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from matplotlib import pyplot as plt

In [0]:
from google.colab import drive 
drive.mount('/mntDrive') 
data=pd.read_csv("/mntDrive/My Drive/НИР магистратура/Оптимизация гиперпараметров алгоритмов градиентного бустинга/Практика/data/input/prepared_facebook_data.csv")

Drive already mounted at /mntDrive; to attempt to forcibly remount, call drive.mount("/mntDrive", force_remount=True).


In [0]:
# grid search
# get some data
X, y = data.loc[:,:-1].values, data.loc[:,-1].values

# build a classifier
clf = xgboost.XGBRegressor(verbosity=0)

param_dist_group_1 = {"max_depth": sp_randint(3, 10),
                  "min_child_weight": sp_randint(1, 10)}

param_dist_group_2 = {"alpha": uniform(loc=0, scale=1),
                  "lambda": uniform(loc=0, scale=1)}

param_dist_group_3 = {"subsample":uniform(loc=0.5, scale=0.4),
                  "colsample_bytree":uniform(loc=0.5, scale=0.4)}


n_iter_search = 243
res=pd.DataFrame([])
# random groups test
for i in range(3):
    #first group:
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist_group_1,
                                      n_iter=n_iter_search, cv=5, scoring="r2", iid=False)

    start = time()
    random_search.fit(X, y)
    print("RandomizedSearchCV for first group took %.2f seconds for %d candidates"
          " parameter settings." % ((time() - start), n_iter_search))


    res_rand_1=pd.DataFrame(random_search.cv_results_)
    res_rand_1['experiment_name']='random search for 1 group'      
    res_rand_1['run_number']=i+1

    clf.set_params(**random_search.best_params_)

    #second group:
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist_group_2,
                                      n_iter=n_iter_search, cv=5, scoring="r2", iid=False)

    start = time()
    random_search.fit(X, y)
    print("RandomizedSearchCV for second group took %.2f seconds for %d candidates"
          " parameter settings." % ((time() - start), n_iter_search))

    res_rand_2=pd.DataFrame(random_search.cv_results_)
    res_rand_2['experiment_name']='random search for 2 group' 
    res_rand_2['run_number']=i+1


    clf.set_params(**random_search.best_params_)

    #third group:
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist_group_3,
                                      n_iter=n_iter_search, cv=5, scoring="r2", iid=False)

    start = time()
    random_search.fit(X, y)
    print("RandomizedSearchCV for third group took %.2f seconds for %d candidates"
          " parameter settings." % ((time() - start), n_iter_search))

    res_rand_3=pd.DataFrame(random_search.cv_results_)
    res_rand_3['experiment_name']='random search for 3 group' 
    res_rand_3['run_number']=i+1


    res=pd.concat([res,res_rand_1,res_rand_2,res_rand_3])

ValueError: ignored

In [0]:
res.to_csv("res_groups.csv",index=False)

NameError: ignored

In [0]:
cp res_groups.csv "/mntDrive/My Drive/НИР магистратура/Оптимизация гиперпараметров алгоритмов градиентного бустинга/Практика/data/results/xgboost_perfomance_group_search.csv"